In [37]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from tabulate import tabulate


# --- 1. Cargar la base de datos
ruta = "C:/Users/Choco/Downloads/base_respondieron_patagonia.csv"
datos = pd.read_csv(ruta, low_memory=False)

# --- 2. Crear variables necesarias
datos['edad'] = pd.to_numeric(datos['CH06'], errors='coerce')
datos['edad2'] = datos['edad'] ** 2
datos['mujer'] = datos['CH04'].replace({1: 0, 2: 1, 9: np.nan})
datos['educ'] = pd.to_numeric(datos['PP07H'], errors='coerce')

# --- 3. Crear variable binaria 'ocupacion'
# (1 = ocupado, 2 = desocupado, los demás se descartan)
datos['ocupacion'] = datos['ESTADO'].replace({1: 1, 2: 0})
datos['ocupacion'] = pd.to_numeric(datos['ocupacion'], errors='coerce')

# --- 4. Mostrar estado general
print("\n🔎 Valores NO nulos por variable:")
print(datos[['edad', 'edad2', 'educ', 'mujer', 'ocupacion']].notnull().sum())

print("\n📌 Valores únicos en 'ocupacion':")
print(datos['ocupacion'].value_counts(dropna=False))

# --- 5. Armar base limpia
modelo = datos[['edad', 'edad2', 'educ', 'mujer', 'ocupacion']].dropna()
print(f"\n📊 Filas válidas después de limpieza: {len(modelo)}")

# --- 6. Separar X e y
X = modelo[['edad', 'edad2', 'educ', 'mujer']]
y = modelo['ocupacion']

# --- 7. División train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=444
)

# --- 8. Tabla de diferencia de medias
media_train = X_train.mean().rename("Media Train")
media_test = X_test.mean().rename("Media Test")
diferencia = (media_train - media_test).rename("Diferencia")

tabla_medias = pd.concat([media_train, media_test, diferencia], axis=1)
print("\n📊 Tabla de diferencia de medias entre Train y Test:")
print(tabla_medias.round(3))
#Cargar la base de datos
ruta = "C:/Users/Choco/Downloads/base_respondieron_patagonia.csv"
datos = pd.read_csv(ruta, low_memory=False)

#Crear variables necesarias
datos['edad'] = pd.to_numeric(datos['CH06'], errors='coerce')
datos['edad2'] = datos['edad'] ** 2
datos['mujer'] = datos['CH04'].replace({1: 0, 2: 1, 9: np.nan})
datos['educ'] = pd.to_numeric(datos['PP07H'], errors='coerce')
datos['ocupacion'] = datos['ESTADO'].replace({1: 1, 2: 0})
datos['ocupacion'] = pd.to_numeric(datos['ocupacion'], errors='coerce')
datos['salario_semanal'] = pd.to_numeric(datos['P21'], errors='coerce')
datos['Hora social'] = pd.to_numeric(datos['CH08'], errors='coerce')
datos[ 'jefe/a' ] = pd.to_numeric(datos['CH16'], errors='coerce')

#Armar base limpia
modelo = datos[['edad', 'edad2', 'educ', 'mujer', 'ocupacion', 'salario_semanal', 'Hora social', 'jefe/a']].dropna(subset=['ocupacion', 'salario_semanal'])

#Separar X e y
X = modelo[['edad', 'edad2', 'educ', 'mujer']]
y = modelo['ocupacion']

#División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=444)

#Estimar modelos de regresión lineal para ocupados
ocupados_train = modelo[(modelo['ocupacion'] == 1) & (modelo.index.isin(X_train.index))]
X_train_ocupados = ocupados_train[['edad', 'edad2', 'educ', 'mujer', 'Hora social', 'jefe/a']]
y_train_ocupados = ocupados_train['salario_semanal']

#Modelos de regresión lineal
modelos = [
    {'nombre': 'Modelo i', 'variables': ['edad']},
    {'nombre': 'Modelo ii', 'variables': ['edad', 'edad2']},
    {'nombre': 'Modelo iii', 'variables': ['edad', 'edad2', 'educ']},
    {'nombre': 'Modelo iv', 'variables': ['edad', 'edad2', 'educ', 'mujer']},
    {'nombre': 'Modelo v', 'variables': ['edad', 'edad2', 'educ', 'mujer', 'Hora social', 'jefe/a']}
]

#Imprimir resultados
variables = set()
for modelo in modelos:
    variables.update(modelo['variables'])
variables = sorted(list(variables))

tabla = []
for variable in variables:
    fila = []
    for modelo in modelos:
        X_modelo = sm.add_constant(X_train_ocupados[modelo['variables']])
        model = sm.OLS(y_train_ocupados, X_modelo).fit()
        params = model.params.round(3)
        se = model.bse.round(2)
        if variable in params.index:
            fila.append(str(params[variable]) + " (" + str(se[variable]) + ")")
        else:
            fila.append("")
    tabla.append([variable] + fila)

headers = ["Variable"] + [modelo['nombre'] for modelo in modelos]
print(tabulate(tabla, headers=headers, tablefmt="grid"))


🔎 Valores NO nulos por variable:
edad         5523
edad2        5523
educ         2407
mujer        5523
ocupacion    5523
dtype: int64

📌 Valores únicos en 'ocupacion':
ocupacion
NaN    3264
1.0    2407
3.0    2357
4.0     644
2.0     115
Name: count, dtype: int64

📊 Filas válidas después de limpieza: 2407

📊 Tabla de diferencia de medias entre Train y Test:
       Media Train  Media Test  Diferencia
edad        41.242      40.638       0.604
edad2     1849.440    1793.362      56.078
educ         0.996       1.011      -0.015
mujer        0.446       0.461      -0.015
+-------------+-------------------+---------------------+---------------------+------------------------+------------------------+
| Variable    | Modelo i          | Modelo ii           | Modelo iii          | Modelo iv              | Modelo v               |
+=============+===================+=====================+=====================+========================+========================+
| Hora social |                 

In [3]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
     confusion_matrix, accuracy_score, roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
df = pd.read_csv('base_respondieron_patagonia.csv')

# Verifica que esté cargado y cómo se llaman las columnas
print(df.columns)
print(df.head())


Index(['nro_hogar', 'componente', 'h15', 'ano4', 'trimestre', 'region',
       'mas_500', 'aglomerado', 'pondera', 'ch03',
       ...
       'GDECIFR', 'PDECIFR', 'ADECIFR', 'DECCFR', 'IDECCFR', 'RDECCFR',
       'GDECCFR', 'PDECCFR', 'ADECCFR', 'PONDIH'],
      dtype='object', length=350)
   nro_hogar componente h15    ano4       trimestre      region mas_500  \
0        1.0        1.0  Sí  2004.0  1er. Trimestre  Patagónica       N   
1        1.0        1.0  Sí  2004.0  1er. Trimestre  Patagónica       N   
2        1.0        2.0  Sí  2004.0  1er. Trimestre  Patagónica       N   
3        1.0        1.0  Sí  2004.0  1er. Trimestre  Patagónica       N   
4        1.0        2.0  Sí  2004.0  1er. Trimestre  Patagónica       N   

                        aglomerado  pondera            ch03  ... GDECIFR  \
0  Comodoro Rivadavia - Rada Tilly    150.0            Jefe  ...     NaN   
1  Comodoro Rivadavia - Rada Tilly    120.0            Jefe  ...     NaN   
2  Comodoro Rivadavia - Rada T

/tmp/ipykernel_3404/2434044526.py:1: DtypeWarning: Columns (1,2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,43,44,45,46,47,48,49,50,55,56,57,58,62,63,64,65,66,67,68,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,99,100,101,102,103,104,105,106,107,108,109,113,114,115,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,141,161,168,171,172,173,182,187,279) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('base_respondieron_patagonia.csv')


In [17]:
# Asegúrate de haber cargado el DataFrame 'df'
import pandas as pd
df = pd.read_csv("base_respondieron_patagonia.csv")

# Verifica los nombres de las columnas
print(df.columns)

Index(['nro_hogar', 'componente', 'h15', 'ano4', 'trimestre', 'region',
       'mas_500', 'aglomerado', 'pondera', 'ch03',
       ...
       'GDECIFR', 'PDECIFR', 'ADECIFR', 'DECCFR', 'IDECCFR', 'RDECCFR',
       'GDECCFR', 'PDECCFR', 'ADECCFR', 'PONDIH'],
      dtype='object', length=350)


/tmp/ipykernel_3404/3961420644.py:3: DtypeWarning: Columns (1,2,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,43,44,45,46,47,48,49,50,55,56,57,58,62,63,64,65,66,67,68,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,99,100,101,102,103,104,105,106,107,108,109,113,114,115,119,120,121,122,123,124,125,126,127,128,130,131,132,133,134,135,141,161,168,171,172,173,182,187,279) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("base_respondieron_patagonia.csv")


In [18]:

# Separar variables predictoras (X) y variables objetivo (y)
X = df.drop(columns='componente') # Usa el nombre exacto que aparece en print(df.columns)
y = df['componente']

In [19]:

# Convertir variables categóricas a númericas
X = pd.get_dummies(X, drop_first=True)

# Eliminar filas con valores faltantes
X = X.dropna()
y = y.loc[X.index] # Alinea y con las filas de X

In [20]:
print("Tamaño de X:", X.shape)
print("Tamaño de y:", y.shape)

Tamaño de X: (0, 3435)
Tamaño de y: (0,)


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=279)



ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.